In [ ]:
import xarray as xr
import os

ds = xr.open_dataset('/home/clein/Documentos/archivos_jupyter/ivt_s_6_2022_Jan.nc')
df = ds.to_dataframe()
print(df)

In [ ]:
import pandas as pd

# Filtrar valores mayores a 250
df_filtered = df.loc[df['IVT'] > 250]

# Resetear el índice y agregar el número como índice al principio
df_filtered.reset_index(inplace=True)
df_filtered.index += 1

# Renombrar el nombre de la columna de índice
df = df_filtered.rename_axis("Index", axis="columns")

# Imprimir el DataFrame
print(df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import math
import xarray as xr
import os

# Convertir la columna de fecha a un objeto de fecha
df['time'] = pd.to_datetime(df['time'])

# Obtener una lista única de fechas
fechas = df['time'].unique()

# Crear una carpeta para guardar los archivos si no existe
carpeta = './datosGraficados'
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

# Definir la latitud y longitud de referencia (para el punto (-180,-90))
lat_ref = -90
lon_ref = -180

# Radio de la Tierra en kilómetros
R = 6371

# Tamaño del píxel en milímetros
tamano_pixel_mm = 0.1

# Iterar sobre las fechas y crear un gráfico para cada una
for i, fecha in enumerate(fechas):
    # Seleccionar los datos correspondientes a la fecha actual
    df_subset = df[df['time'] == fecha]

    # Filtrar los objetos azules
    objetos_azules = df_subset

    if objetos_azules.empty:
        continue

    # Obtener los valores de latitud y longitud para los objetos azules
    latitudes_azules = objetos_azules['latitude']
    longitudes_azules = objetos_azules['longitude']

    # Resto del código para crear el gráfico
    fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.scatter(longitudes_azules, latitudes_azules, c='blue', s=5, transform=ccrs.PlateCarree())
    ax.coastlines()

    # Agregar líneas de latitud y longitud
    ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--', xlocs=range(-180,181,10), ylocs=range(-90,91,10))

    # Agregar características al mapa
    ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='gray')
    ax.add_feature(cfeature.RIVERS, linewidth=0.5, edgecolor='blue')

    # Ajustar el tamaño de la figura al límite del contenido
    plt.tight_layout()

    # Guardar el gráfico con el nombre de la fecha correspondiente
    fecha_py = pd.to_datetime(fecha).to_pydatetime()
    nombre_archivo = fecha_py.strftime('%Y-%m-%d') + '.png'
    archivo = os.path.join(carpeta, nombre_archivo)
    plt.savefig(archivo, dpi=fig.dpi)

    # Imprimir el nombre del archivo
    print("Nombre de la imagen:", nombre_archivo)

    # Imprimir la resolución de la imagen
    resolucion = (fig.dpi * fig.get_size_inches()).astype(int)
    print("Resolución de la imagen (ancho x alto):", resolucion)

    # Calcular el tamaño del píxel en milímetros
    tamano_pixel = (1 / fig.dpi) * 25.4
    print("Tamaño del píxel:", tamano_pixel, "mm")

    # Imprimir la escala de la imagen
    escala = ax.get_xlim()[1] - ax.get_xlim()[0]
    print("Escala de la imagen:", escala)

    # Imprimir información adicional de la imagen
    print("Otras propiedades geoespaciales:")
    # - Proyección cartográfica
    print("- Proyección cartográfica: ", ax.projection)
    # - Sistema de coordenadas de referencia (CRS)
    print("- CRS: ", ax.projection.proj4_init)
    # - Extensión espacial
    print("- Extensión espacial (latitud mínima, latitud máxima, longitud mínima, longitud máxima):",
          ax.get_extent())
    # - Unidades de medida
    print("- Unidades de medida: grados")

    # Cerrar la figura para liberar memoria
    plt.close(fig)

print("Gráficos guardados.")
